In [1]:
%load_ext autoreload
%autoreload 2
#import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
import keras
from tqdm import tqdm, tqdm_pandas
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
import csv
warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keppelWord = ['keppel corporation', 'keppel capital', 'keppel reit', 'keppel infrastructure','keppel o&m', 'keppel energy', 'keppel offshore', 'keppel dhcs', 'keppel fairway', 'keppel professorship', 'keppel reit', 'keppel group', 'keppel land' , 'keppel singmarine', 'keppel fels', 'keppel shipyard', 'keppel gas','keppel t&t', 'keppel bay', 'keppel telecom','keppel corp']
prudentialWord = ['prudential tower', 'prudential assurance', 'prudential financial', 'prudential group','prudential insurance','prudential life','prudential alliance','prudential capital']

In [5]:
df_files = pd.read_csv("safe/df_file_final.csv", lineterminator='\n')

In [6]:
df_files.head()

,Unnamed: 0,Unnamed: 0.1,id,filename,company,file_path,sentiment,sentences,title,occurence,frequency
0,0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,"['', '', 'is a premier asset manager in Asia. ...",\r,10,1.557239
1,1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,['TP is lifted to \nS$6.00 as we roll over val...,()\r,43,4.946321
2,2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,"['', '. \n\n \n19 March 2010, Baku, Azerbaijan...",(Microsoft Word - SOCARKEPPEL.doc)\r,20,2.789354
3,3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,[],\r,0,0.000000
4,4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,['/o Servoo Apportorai \n(Tiong Seng Contracto...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r,24,3.245119


In [7]:
def doWeDrop(sentences):
    isValid = False
    for phrase in sentences:
        lowered = phrase.lower()
        #print(lowered)
        for expr1 in keppelWord:
            if(expr1 in lowered):
                #print("valid")
                isValid = True
        for expr2 in prudentialWord:
            if(expr2 in lowered):
                #print("valid")
                isValid = True
    return not isValid

In [25]:
def find_sentiment(important_sents,sentimentAnalyzer):
    try:
        #text = extract_text.apply(path)
        #important_sents = text_summarization.apply(text)
        #print(len(important_sents))
        #if(doWeDrop(important_sents)):
            #return -500
        importance = np.mean(sentimentAnalyzer.predict(important_sents))
        return np.mean(importance)
    except:
        return -999

In [9]:
## Transform into array else loaded as string
from ast import literal_eval
df_files.sentences = df_files.sentences.apply(literal_eval)

In [26]:
#model = keras.models.load_model('models/my_amazon_model.h5')
analyzser = sentiment_analyzer.SentimentAnalyzer('models/my_amazon_model.h5')
tqdm().pandas(desc="Progress:")
df_files.sentiment = df_files.progress_apply(lambda row: np.mean(find_sentiment(row['sentences'],analyzser)),axis=1)

10000


0it [00:00, ?it/s]
Progress::   1%|          | 4/387 [00:01<02:54,  2.20it/s]
0it [00:00, ?it/s]
 94%|█████████▎| 567/606 [00:02<00:00, 228.27it/s]


Progress::   7%|▋         | 28/387 [00:31<08:22,  1.40s/it]
0it [00:00, ?it/s]
Progress::  13%|█▎        | 51/387 [00:52<05:52,  1.05s/it]
0it [00:00, ?it/s]
Progress::  14%|█▎        | 53/387 [00:53<04:08,  1.34it/s]
0it [00:00, ?it/s]
Progress::  14%|█▍        | 56/387 [00:55<03:08,  1.76it/s]


Progress::  15%|█▍        | 57/387 [00:55<02:59,  1.84it/s]
0it [00:00, ?it/s]
Progress::  17%|█▋        | 66/387 [01:05<06:44,  1.26s/it]
0it [00:00, ?it/s]
Progress::  18%|█▊        | 71/387 [01:11<08:20,  1.58s/it]
0it [00:00, ?it/s]
  0%|          | 0/134 [00:00<?, ?it/s]


 79%|███████▉  | 263/332 [00:01<00:00, 211.60it/s]


...

Progress::  42%|████▏     | 164/387 [02:46<03:55,  1.06s/it]
0it [00:00, ?it/s]
Progress::  45%|████▌     | 175/387 [02:54<02:17,  1.55it/s]
0it [00:00, ?it/s]
Progress::  46%|████▌     | 178/387 [02:56<01:43,  2.02it/s]
0it [00:00, ?it/s]
Progress::  47%|████▋     | 182/387 [02:59<02:37,  1.30it/s]
0it [00:00, ?it/s]
Progress::  48%|████▊     | 184/387 [03:03<05:24,  1.60s/it]
0it [00:00, ?it/s]
Progress::  49%|████▉     | 190/387 [03:10<04:34,  1.39s/it]
0it [00:00, ?it/s]
 32%|███▏      | 67/208 [00:00<00:00, 219.66it/s]


Progress::  51%|█████     | 198/387 [03:16<01:57,  1.61it/s]
0it [00:00, ?it/s]
Progress::  53%|█████▎    | 204/387 [03:19<01:33,  1.97it/s]
0it [00:00, ?it/s]
Progress::  53%|█████▎    | 206/387 [03:20<01:31,  1.98it/s]
0it [00:00, ?it/s]
Progress::  54%|█████▍    | 209/387 [03:21<01:20,  2.22it/s]
0it [00:00, ?it/s]
Progress::  54%|█████▍    | 210/387 [03:21<01:17,  2.27it/s]
0it [00:00, ?it/s]
Progress::  58%|█████▊    | 223/387 [03:29<01:28,  1.85it/s]
0it [00:00, ?it/s]
Progress::  58%|█████▊    | 226/387 [03:30<01:18,  2.06it/s]
0it [00:00, ?it/s]
Progress::  59%|█████▊    | 227/387 [03:31<01:14,  2.15it/s]
0it [00:00, ?it/s]
Progress::  59%|█████▉    | 230/387 [03:32<01:13,  2.14it/s]
0it [00:00, ?it/s]
Progress::  60%|█████▉    | 231/387 [03:32<01:10,  2.21it/s]
0it [00:00, ?it/s]
Progress::  61%|██████▏   | 238/387 [03:38<01:46,  1.41it/s]
0it [00:00, ?it/s]
Progress::  63%|██████▎   | 245/387 [03:41<01:11,  1.99it/s]
0it [00:00, ?it/s]
Progress::  64%|██████▍   | 249/387 [03:

Progress::  67%|██████▋   | 258/387 [03:47<00:57,  2.23it/s]
0it [00:00, ?it/s]
Progress::  68%|██████▊   | 265/387 [03:51<01:03,  1.93it/s]
0it [00:00, ?it/s]
Progress::  70%|██████▉   | 270/387 [03:53<00:56,  2.07it/s]
0it [00:00, ?it/s]
Progress::  71%|███████   | 275/387 [03:56<01:03,  1.78it/s]
0it [00:00, ?it/s]
Progress::  71%|███████▏  | 276/387 [03:57<00:57,  1.94it/s]
0it [00:00, ?it/s]
Progress::  78%|███████▊  | 300/387 [04:10<00:50,  1.72it/s]
0it [00:00, ?it/s]
Progress::  79%|███████▊  | 304/387 [04:12<00:47,  1.76it/s]
0it [00:00, ?it/s]
Progress::  79%|███████▉  | 305/387 [04:12<00:42,  1.92it/s]
0it [00:00, ?it/s]
Progress::  80%|████████  | 310/387 [04:15<00:40,  1.92it/s]
0it [00:00, ?it/s]
Progress::  81%|████████  | 313/387 [04:16<00:36,  2.03it/s]
0it [00:00, ?it/s]
Progress::  83%|████████▎ | 320/387 [04:20<00:31,  2.15it/s]
0it [00:00, ?it/s]
Progress::  83%|████████▎ | 323/387 [04:21<00:28,  2.28it/s]


Progress::  84%|████████▍ | 327/387 [04:23<00:29,  2.00it/s]
0it [00:00, ?it/s]
Progress::  86%|████████▌ | 331/387 [04:26<00:41,  1.34it/s]
0it [00:00, ?it/s]
Progress::  86%|████████▋ | 334/387 [04:28<00:32,  1.63it/s]
0it [00:00, ?it/s]
Progress::  89%|████████▊ | 343/387 [04:34<00:29,  1.51it/s]
0it [00:00, ?it/s]
Progress::  95%|█████████▍| 367/387 [04:46<00:09,  2.02it/s]
0it [00:00, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 150.66it/s]


In [27]:
df_files

,Unnamed: 0,Unnamed: 0.1,id,filename,company,file_path,sentiment,sentences,title,occurence,frequency
0,0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,0.623874,"[, , is a premier asset manager in Asia. It is...",\r,10,1.557239
1,1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,0.600225,[TP is lifted to \nS$6.00 as we roll over valu...,()\r,43,4.946321
2,2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,0.668060,"[, . \n\n \n19 March 2010, Baku, Azerbaijan\n ...",(Microsoft Word - SOCARKEPPEL.doc)\r,20,2.789354
3,3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,-999.000000,[],\r,0,0.000000
4,4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,0.588251,[/o Servoo Apportorai \n(Tiong Seng Contractor...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r,24,3.245119
5,5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,0.644867,[\nCONTROL OF NAVIGATION\n28 Collision Regulat...,\r,12,1.249041
6,6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,0.735468,"[, AT THE 2003 SECOND QUARTER RESULTS PRESENT...",(Microsoft Word - KOMTechnology StrategyJuly03...,21,2.694096
7,7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,0.611130,[Unitholdings\n158 \n Corporate Information\n1...,\r,668,59.796418
8,8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,0.652324,[Kingcheung. \nWorking experience and \noccup...,(http://info.sgx.com/webcorannc.nsf/vwprint_po...,4,0.554044
9,9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,0.805171,[John) \nLYCA PT747.S6 B4 1965 A history of th...,\r,1,0.106658


In [28]:
def getSubmission(df):
    dfCleaned = df[df['sentiment']>0.01]
    top_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=False).head(10).id)
    down_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=True).head(10).id)
    top_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=False).head(10).id)
    down_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=True).head(10).id)
    return top_keppel,down_keppel, top_prud, down_prud

In [29]:
def createSubCSV(topId_k,downId_k,topId_p,downId_p):
    with open("sub.csv",'w',newline='') as csvfile:
        spamwritter = csv.writer(csvfile)
        spamwritter.writerow(['id','rank'])
        for i in range(len(df_files.sentiment)):
            if(i in topId_k):
                spamwritter.writerow([i,(10-topId_k.index(i))])
            elif(i in downId_k):
                spamwritter.writerow([i,(downId_k.index(i)-10)])
            elif(i in topId_p):
                spamwritter.writerow([i,(10-topId_p.index(i))])
            elif(i in downId_p):
                spamwritter.writerow([i,(downId_p.index(i)-10)])
            else:
                spamwritter.writerow([i,0])

In [30]:
top_k,down_k, top_p, down_p = getSubmission(df_files)
createSubCSV(top_k,down_k, top_p, down_p)

In [37]:
##Doesn't change anything
topT,downT = getSubmission(NEdf)
createSubCSV(topT,downT)